<a href="https://colab.research.google.com/github/yilinmiao/genai-solution/blob/main/sentiment_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [20]:
import os
from google.colab import userdata

os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')
os.environ["OPENAI_API_BASE"] = "https://api.openai.com/v1"

In [21]:
!pip install langchain-community

In [22]:
from langchain.chat_models import ChatOpenAI # this is the new import statement
from langchain.prompts import PromptTemplate
from langchain.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field, NonNegativeInt
from typing import List
from random import sample
from langchain.document_loaders.csv_loader import CSVLoader

loader = CSVLoader(file_path='./tv-reviews.csv')
data = loader.load()

print(data)


[Document(metadata={'source': './tv-reviews.csv', 'row': 0}, page_content="TV Name: Imagix Pro\nReview Title: Amazing Picture Quality\nReview Rating: 9\nReview Text: I recently purchased the Imagix Pro and I am blown away by its picture quality. The colors are vibrant and the images are crystal clear. It feels like I'm watching movies in a theater! The sound is also impressive, creating a truly immersive experience. Highly recommended!"), Document(metadata={'source': './tv-reviews.csv', 'row': 1}, page_content="TV Name: Imagix Pro\nReview Title: Impressive Features\nReview Rating: 8\nReview Text: The Imagix Pro is packed with impressive features that enhance my viewing experience. The smart functionality allows me to easily stream my favorite shows and movies. The remote control is user-friendly and has convenient shortcuts. The slim design is sleek and fits perfectly in my living room. The only downside is that the sound could be better, but overall, I'm satisfied."), Document(metadat

In [23]:

model_name = 'gpt-3.5-turbo'
# llm = OpenAI(model_name=model_name, temperature=0) - This has been replaced with the next line of code
llm = ChatOpenAI()

class ReviewSentiment(BaseModel):
    positives: List[NonNegativeInt] = Field(description="index of a positive TV review, starting from 0")
    negatives: List[NonNegativeInt] = Field(description="index of a negative TV review, starting from 0")

parser = PydanticOutputParser(pydantic_object=ReviewSentiment)
print(parser.get_format_instructions())


The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"positives": {"description": "index of a positive TV review, starting from 0", "items": {"minimum": 0, "type": "integer"}, "title": "Positives", "type": "array"}, "negatives": {"description": "index of a negative TV review, starting from 0", "items": {"minimum": 0, "type": "integer"}, "title": "Negatives", "type": "array"}}, "required": ["positives", "negatives"]}
```


In [24]:

prompt = PromptTemplate(
    template="{question}\n{format_instructions}\nContext: {context}",
    input_variables=["question", "context"],
    partial_variables={"format_instructions": parser.get_format_instructions},
)
question = """
    Classify TV reviews provided in the context into positive and negative.
    Only use the reviews provided in this context, do not make up new reviews or use any existing information you know about these TVs.
    If there are no positive or negative reviews, output an empty JSON array.
"""

reviews_to_classify = sample(data, 3)
context = '\n'.join(review.page_content for review in reviews_to_classify)

query = prompt.format(context = context, question = question)
print(query)



    Classify TV reviews provided in the context into positive and negative. 
    Only use the reviews provided in this context, do not make up new reviews or use any existing information you know about these TVs. 
    If there are no positive or negative reviews, output an empty JSON array. 

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"positives": {"description": "index of a positive TV review, starting from 0", "items": {"minimum": 0, "type": "integer"}, "title": "Positives", "type": "array"}, "negatives": {"description": "index of a negative TV review, starting from 

In [25]:
output = llm.predict(query) #the "predict" has been added
print(output)
result = parser.parse(output)
print(result)
print("Positives:\n" + "\n".join([reviews_to_classify[i].page_content for i in result.positives]))
print("Negatives:\n" + "\n".join([reviews_to_classify[i].page_content for i in result.negatives]))

{
    "positives": [1, 2],
    "negatives": [0]
}
positives=[1, 2] negatives=[0]
Positives:
TV Name: VisionMax Ultra
Review Title: Excellent Picture Clarity
Review Rating: 9
Review Text: The picture clarity of the VisionMax Ultra is simply outstanding. The images are sharp, and the colors are vivid. Whether I'm watching movies, sports, or playing games, the visuals are always stunning. This TV takes my viewing experience to a whole new level!
TV Name: Imagix Pro
Review Title: Stunning Design
Review Rating: 10
Review Text: The Imagix Pro not only offers exceptional performance but also boasts a stunning design. The ultra-thin bezels and sleek finish make it a focal point of my living room. The TV blends seamlessly with my decor. It's a true work of art!
Negatives:
TV Name: VisionMax Ultra
Review Title: Disappointing Sound
Review Rating: 5
Review Text: While the picture quality of the VisionMax Ultra is exceptional, the sound quality falls short. The built-in speakers lack depth and the 